This is a collection of observations around why MIDI is not training well.

In [60]:
%load_ext autoreload
%autoreload 2
%cd ../MusicTransformer-tensorflow2.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/notebooks/src/MusicTransformer-tensorflow2.0


# 1. Comparing Datasets

I first worked through the local GPU issues, and then set it up to train `classic_piano` in comparison to `midi`. I graphed the metrics through Tensorboard.

In [ ]:
from train import train
train(
    '/data/classic_piano_preprocessed',
    './save',
    log_dir='/tmp/logs/classic_piano_preprocessed')

In [ ]:
from train import train
train(
    '../out/transformer-preprocess',
    './save-transformer-preprocess',
    log_dir='/tmp/logs/transformer-preprocess'
)

Below is a graph of the accuracy, with `classic_piano` through ~65 epochs and `midi` through ~20 epochs. `classic_piano` is clearly on the upswing, and looks [comparable in slope to the results in the original repo](https://github.com/jason9693/MusicTransformer-tensorflow2.0#result).

`midi` is going nowhere fast (which is why I stopped it early).

![Accuracy](./accuracy.png)

Here's a graph of the loss. The loss for `midi` stays at 0.

![Loss](./loss.png)

The most likely culprit is that there's something wrong with the underlying data. So let's look at that.

# 2. Examining the Data

Let's log out the tensor information we're getting from each dataset.

In [68]:
from data import Data
import tensorflow as tf

batch_size = 2
max_seq = 1024

input_paths = [
    '/data/classic_piano_preprocessed',
    '../out/transformer-preprocess',    
]

for input_path in input_paths:
    print('\n\nInput path: {}'.format(input_path))
    dataset = Data(input_path)
    batch_x, batch_y = dataset.slide_seq2seq_batch(batch_size, max_seq)
    print('batch_x')
    print(batch_x.shape)
    print(batch_x[0:5])
    print('min', tf.math.reduce_min(batch_x, axis=None, keepdims=False, name=None))
    print('max', tf.math.reduce_max(batch_x, axis=None, keepdims=False, name=None))
    print('mean', tf.math.reduce_mean(batch_x, axis=None, keepdims=False, name=None))
    print('batch_y')
    print(batch_y.shape)
    print(batch_y)
    print('min', tf.math.reduce_min(batch_y, axis=None, keepdims=False, name=None))
    print('max', tf.math.reduce_max(batch_y, axis=None, keepdims=False, name=None))
    print('mean', tf.math.reduce_mean(batch_y, axis=None, keepdims=False, name=None))



Input path: /data/classic_piano_preprocessed
batch_x
(2, 1024)
[[367  51 256 ... 304 163 361]
 [369  79 281 ...  75 366  60]]
min tf.Tensor(27, shape=(), dtype=int64)
max tf.Tensor(380, shape=(), dtype=int64)
mean tf.Tensor(218, shape=(), dtype=int64)
batch_y
(2, 1024)
[[ 51 256 168 ... 163 361  40]
 [ 79 281 179 ... 366  60 308]]
min tf.Tensor(27, shape=(), dtype=int64)
max tf.Tensor(380, shape=(), dtype=int64)
mean tf.Tensor(218, shape=(), dtype=int64)


Input path: ../out/transformer-preprocess
batch_x
(2, 1024)
[[390. 388. 388. ... 388. 388. 388.]
 [390. 388. 388. ... 388. 388. 388.]]
min tf.Tensor(388.0, shape=(), dtype=float64)
max tf.Tensor(390.0, shape=(), dtype=float64)
mean tf.Tensor(388.001953125, shape=(), dtype=float64)
batch_y
(2, 1024)
[[388. 388. 388. ... 388. 388. 388.]
 [388. 388. 388. ... 388. 388. 388.]]
min tf.Tensor(388.0, shape=(), dtype=float64)
max tf.Tensor(388.0, shape=(), dtype=float64)
mean tf.Tensor(388.0, shape=(), dtype=float64)


The shapes of each tensor batch are identical, but the data is pretty different. For `classic_piano`, there's a pretty wide range of values (27-380), but for `midi`, they all seem to fall in a small band (`388-390`).

That seems pretty suspicious.

# 3. Preprocessing MIDI step

Next let's see how the data is being preprocessed by in the original repo.

In [5]:
# Download the classic_piano dataset

!sh /src/dataset/scripts/classic_piano_downloader.sh /data/classic_piano

Pages: albeniz.htm bach.htm balak.htm beeth.htm borodin.htm brahms.htm burgm.htm chopin.htm clementi.htm debuss.htm godowsky.htm grana.htm grieg.htm haydn.htm liszt.htm mendelssohn.htm moszkowski.htm mozart.htm muss.htm rach.htm ravel.htm schub.htm schum.htm sinding.htm tschai.htm xmas.htm
http://www.piano-midi.de/midis/albeniz/alb_esp1_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_esp2_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_esp3_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_esp4_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_esp5_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_esp6_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_se1_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_se2_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_se3_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_se4_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_se5_format0.mid
http://www.piano-midi.de/midis/albeniz/alb_se6_format0

HTTP request sent, awaiting response... 200 OK
Length: 15447 (15K) [audio/midi]
Saving to: ‘/data/classic_piano/bach_847_format0.mid’

bach_847_format0.mi 100%[===================>]  15.08K  --.-KB/s    in 0.001s  

2020-05-25 19:50:07 (25.7 MB/s) - ‘/data/classic_piano/bach_847_format0.mid’ saved [15447/15447]

--2020-05-25 19:50:07--  http://www.piano-midi.de/midis/bach/bach_850_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 11881 (12K) [audio/midi]
Saving to: ‘/data/classic_piano/bach_850_format0.mid’

bach_850_format0.mi 100%[===================>]  11.60K  --.-KB/s    in 0s      

2020-05-25 19:50:08 (68.6 MB/s) - ‘/data/classic_piano/bach_850_format0.mid’ saved [11881/11881]

FINISHED --2020-05-25 19:50:08--
Total wall clock time: 0.5s
Downloaded: 3 files, 39K in 0.001s (41.1 MB/s)
http://www.piano-midi.de/midis/balakirew/islamei_format0.mid
--2020-05-25 19:50:08--  http://www.piano-midi.de/midis/balakirew/is

Length: 30199 (29K) [audio/midi]
Saving to: ‘/data/classic_piano/beethoven_opus22_4_format0.mid’

beethoven_opus22_4_ 100%[===================>]  29.49K  --.-KB/s    in 0.002s  

2020-05-25 19:50:10 (17.8 MB/s) - ‘/data/classic_piano/beethoven_opus22_4_format0.mid’ saved [30199/30199]

--2020-05-25 19:50:10--  http://www.piano-midi.de/midis/beethoven/mond_1_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 14768 (14K) [audio/midi]
Saving to: ‘/data/classic_piano/mond_1_format0.mid’

mond_1_format0.mid  100%[===================>]  14.42K  --.-KB/s    in 0s      

2020-05-25 19:50:10 (53.8 MB/s) - ‘/data/classic_piano/mond_1_format0.mid’ saved [14768/14768]

--2020-05-25 19:50:10--  http://www.piano-midi.de/midis/beethoven/mond_2_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 9508 (9.3K) [audio/midi]
Saving to: ‘/data/classic_piano/mond_2_format0.

HTTP request sent, awaiting response... 200 OK
Length: 45653 (45K) [audio/midi]
Saving to: ‘/data/classic_piano/beethoven_hammerklavier_3_format0.mid’

beethoven_hammerkla 100%[===================>]  44.58K  --.-KB/s    in 0.003s  

2020-05-25 19:50:12 (14.9 MB/s) - ‘/data/classic_piano/beethoven_hammerklavier_3_format0.mid’ saved [45653/45653]

--2020-05-25 19:50:12--  http://www.piano-midi.de/midis/beethoven/beethoven_hammerklavier_4_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 82201 (80K) [audio/midi]
Saving to: ‘/data/classic_piano/beethoven_hammerklavier_4_format0.mid’

beethoven_hammerkla 100%[===================>]  80.27K  --.-KB/s    in 0.006s  

2020-05-25 19:50:12 (12.6 MB/s) - ‘/data/classic_piano/beethoven_hammerklavier_4_format0.mid’ saved [82201/82201]

FINISHED --2020-05-25 19:50:12--
Total wall clock time: 3.7s
Downloaded: 29 files, 1.0M in 0.07s (13.8 MB/s)
http://www.piano-midi.de/midis/borodin

HTTP request sent, awaiting response... 200 OK
Length: 13059 (13K) [audio/midi]
Saving to: ‘/data/classic_piano/brahms_opus117_2_format0.mid’

brahms_opus117_2_fo 100%[===================>]  12.75K  --.-KB/s    in 0s      

2020-05-25 19:50:15 (28.5 MB/s) - ‘/data/classic_piano/brahms_opus117_2_format0.mid’ saved [13059/13059]

--2020-05-25 19:50:15--  http://www.piano-midi.de/midis/brahms/br_rhap_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 28882 (28K) [audio/midi]
Saving to: ‘/data/classic_piano/br_rhap_format0.mid’

br_rhap_format0.mid 100%[===================>]  28.21K  --.-KB/s    in 0.002s  

2020-05-25 19:50:15 (15.0 MB/s) - ‘/data/classic_piano/br_rhap_format0.mid’ saved [28882/28882]

FINISHED --2020-05-25 19:50:15--
Total wall clock time: 1.3s
Downloaded: 10 files, 239K in 0.01s (16.5 MB/s)
http://www.piano-midi.de/midis/burgmueller/burg_perlen_format0.mid
http://www.piano-midi.de/midis/burgmueller/bur

...

HTTP request sent, awaiting response... 302 Found
Location: http://www.piano-midi.de/ [following]
--2020-05-25 19:50:50--  http://www.piano-midi.de/
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/data/classic_piano/rav_scarbo_format0.mid’

rav_scarbo_format0.     [ <=>                ]  10.89K  --.-KB/s    in 0.004s  

2020-05-25 19:50:50 (2.94 MB/s) - ‘/data/classic_piano/rav_scarbo_format0.mid’ saved [11154]

FINISHED --2020-05-25 19:50:50--
Total wall clock time: 1.4s
Downloaded: 5 files, 75K in 0.02s (4.18 MB/s)
http://www.piano-midi.de/midis/schubert/schubert_D850_1_format0.mid
http://www.piano-midi.de/midis/schubert/schubert_D850_2_format0.mid
http://www.piano-midi.de/midis/schubert/schubert_D850_3_format0.mid
http://www.piano-midi.de/midis/schubert/schubert_D850_4_format0.mid
http://www.piano-midi.de/midis/schubert/schub_d760_1_format0.mid
http://www.piano-midi.de/midis/schubert/sch

HTTP request sent, awaiting response... 200 OK
Length: 19018 (19K) [audio/midi]
Saving to: ‘/data/classic_piano/schumm-5_format0.mid’

schumm-5_format0.mi 100%[===================>]  18.57K  --.-KB/s    in 0.001s  

2020-05-25 19:50:52 (21.8 MB/s) - ‘/data/classic_piano/schumm-5_format0.mid’ saved [19018/19018]

--2020-05-25 19:50:52--  http://www.piano-midi.de/midis/schubert/schumm-6_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 19301 (19K) [audio/midi]
Saving to: ‘/data/classic_piano/schumm-6_format0.mid’

schumm-6_format0.mi 100%[===================>]  18.85K  --.-KB/s    in 0.001s  

2020-05-25 19:50:52 (16.5 MB/s) - ‘/data/classic_piano/schumm-6_format0.mid’ saved [19301/19301]

--2020-05-25 19:50:52--  http://www.piano-midi.de/midis/schubert/schu_143_1_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 55340 (54K) [audio/midi]
Saving to: ‘

HTTP request sent, awaiting response... 200 OK
Length: 56643 (55K) [audio/midi]
Saving to: ‘/data/classic_piano/schub_d960_4_format0.mid’

schub_d960_4_format 100%[===================>]  55.32K  --.-KB/s    in 0.004s  

2020-05-25 19:50:54 (14.1 MB/s) - ‘/data/classic_piano/schub_d960_4_format0.mid’ saved [56643/56643]

FINISHED --2020-05-25 19:50:54--
Total wall clock time: 4.3s
Downloaded: 29 files, 1.1M in 0.2s (5.97 MB/s)
http://www.piano-midi.de/midis/schumann/schum_abegg_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_1_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_2_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_3_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_4_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_5_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_6_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_7_format0.mid
http://www.piano-midi.de/midis/schumann/scn15_8_format0.mid
http://www.piano-midi.de/m

HTTP request sent, awaiting response... 200 OK
Length: 19412 (19K) [audio/midi]
Saving to: ‘/data/classic_piano/scn16_1_format0.mid’

scn16_1_format0.mid 100%[===================>]  18.96K  --.-KB/s    in 0.001s  

2020-05-25 19:50:57 (34.4 MB/s) - ‘/data/classic_piano/scn16_1_format0.mid’ saved [19412/19412]

--2020-05-25 19:50:57--  http://www.piano-midi.de/midis/schumann/scn16_2_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 35748 (35K) [audio/midi]
Saving to: ‘/data/classic_piano/scn16_2_format0.mid’

scn16_2_format0.mid 100%[===================>]  34.91K  --.-KB/s    in 0.002s  

2020-05-25 19:50:57 (16.3 MB/s) - ‘/data/classic_piano/scn16_2_format0.mid’ saved [35748/35748]

--2020-05-25 19:50:57--  http://www.piano-midi.de/midis/schumann/scn16_3_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 23685 (23K) [audio/midi]
Saving to: ‘/data/cl

HTTP request sent, awaiting response... 200 OK
Length: 10306 (10K) [audio/midi]
Saving to: ‘/data/classic_piano/ty_juli_format0.mid’

ty_juli_format0.mid 100%[===================>]  10.06K  --.-KB/s    in 0s      

2020-05-25 19:50:59 (41.0 MB/s) - ‘/data/classic_piano/ty_juli_format0.mid’ saved [10306/10306]

--2020-05-25 19:50:59--  http://www.piano-midi.de/midis/tchaikovsky/ty_august_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 24193 (24K) [audio/midi]
Saving to: ‘/data/classic_piano/ty_august_format0.mid’

ty_august_format0.m 100%[===================>]  23.63K  --.-KB/s    in 0.001s  

2020-05-25 19:50:59 (21.4 MB/s) - ‘/data/classic_piano/ty_august_format0.mid’ saved [24193/24193]

--2020-05-25 19:50:59--  http://www.piano-midi.de/midis/tchaikovsky/ty_september_format0.mid
Reusing existing connection to www.piano-midi.de:80.
HTTP request sent, awaiting response... 200 OK
Length: 19512 (19K) [audio/midi]
Sav

In [8]:
import os

CLASSIC_PIANO_DATA = '/data/classic_piano'
MIDI = '../out/midi'

print('{} files in {}'.format(len(os.listdir(CLASSIC_PIANO_DATA)), 'classic_piano'))
print('{} files in {}'.format(len(os.listdir(MIDI)), 'MIDI'))

329 files in classic_piano
156 files in MIDI


As an aside, the sample dataset has 329 files, while we have 156; but I'm not sure how best to compare the lengths of the files. It may be worth measuring more definitively how our dataset compares to `classic_piano` or one of the other sample ones.

We can try running `preprocess` on the different datasets and see if anything jumps out as being fishy.

In [70]:
from preprocess import preprocess_midi_files_under_original

preprocess_midi_files_under_original(CLASSIC_PIANO_DATA, '/tmp/data/classic_piano_preprocessed')
preprocess_midi_files_under_original(MIDI, '/tmp/data/midi')

Everything seems legit. Let's try pre-processing individual files

In [71]:
from preprocess import preprocess_midi_original
import os

def printPreprocessedData(file):
    print(file)
    print(preprocess_midi_original(file))

classic_piano_data = ['{folder}/{file}'.format(folder=CLASSIC_PIANO_DATA, file=file) for file in os.listdir(CLASSIC_PIANO_DATA)]
midi_data = [os.path.abspath('{folder}/{file}'.format(folder=MIDI, file=file)) for file in os.listdir(MIDI)]

In [26]:
printPreprocessedData(classic_piano_data[2])

/data/classic_piano/schuim-4_format0.mid
[366, 83, 366, 44, 265, 364, 87, 211, 264, 366, 83, 172, 264, 364, 80, 265, 208, 366, 80, 366, 63, 364, 59, 364, 56, 265, 211, 364, 83, 265, 208, 366, 80, 265, 364, 75, 265, 203, 366, 75, 265, 208, 364, 80, 265, 203, 366, 75, 265, 364, 71, 265, 199, 366, 71, 265, 203, 364, 75, 265, 199, 366, 71, 265, 364, 68, 265, 196, 366, 68, 265, 199, 364, 71, 265, 196, 366, 68, 265, 191, 364, 63, 265, 191, 364, 63, 363, 58, 363, 67, 365, 70, 363, 51, 260, 215, 187, 184, 211, 208, 203, 199, 196, 274, 191, 186, 195, 198, 179, 279, 366, 83, 366, 44, 265, 211, 364, 87, 265, 172, 366, 83, 265, 364, 80, 265, 208, 366, 80, 366, 63, 364, 59, 364, 56, 265, 211, 364, 83, 265, 208, 366, 80, 265, 364, 75, 265, 203, 366, 75, 265, 208, 364, 80, 265, 203, 366, 75, 265, 364, 71, 265, 199, 366, 71, 265, 203, 364, 75, 265, 199, 366, 71, 265, 364, 68, 265, 196, 366, 68, 265, 199, 364, 71, 265, 196, 366, 68, 265, 191, 364, 63, 265, 191, 363, 58, 363, 51, 363, 67, 365, 70, 364, 

In [33]:
for i in range(0, 5):
    printPreprocessedData(midi_data[i])

/notebooks/src/out/midi/liquidmind-10-thejoyofquietpt2.wav.mid
[]
/notebooks/src/out/midi/gas-03-nach1912.wav.mid
[]
/notebooks/src/out/midi/easternawakening-14-hillsofnepal.wav.mid
[]
/notebooks/src/out/midi/rudyadrian-06-thelegendofkristylynn.wav.mid
[]
/notebooks/src/out/midi/deuter-05-thesource.wav.mid
[]


Very interesting! So what jumps out here is that the `MIDI` files are being pre-processed as empty arrays. This would go a good way towards explaining why the incoming tensors in the training step look so fishy.

Why are they failing to be preprocessed? Let's crack open `midi_processor` and see what `encode_midi` is doing. Below is a reproduction of some of the code from the original `midi_processor/processor.py` file:

In [74]:
import pretty_midi

midi_file_one = pretty_midi.PrettyMIDI(midi_file=midi_data[0])
piano_file_one = pretty_midi.PrettyMIDI(midi_file=classic_piano_data[0])

from midi_processor.processor import _control_preprocess, _note_preprocess, _divide_note, _make_time_sift_events, _snote2events

def _note_preprocess_jupyter(susteins, notes):
    note_stream = []

    for sustain in susteins:
        for note_idx, note in enumerate(notes):
            if note.start < sustain.start:
                note_stream.append(note)
            elif note.start > sustain.end:
                notes = notes[note_idx:]
                sustain.transposition_notes()
                break
            else:
                sustain.add_managed_note(note)

    for sustain in susteins:
        note_stream += sustain.managed_notes

    note_stream.sort(key= lambda x: x.start)
    return note_stream

def encode_midi_jupyter(file_path):
    events = []
    notes = []
    mid = pretty_midi.PrettyMIDI(midi_file=file_path)

    for inst in mid.instruments:
        inst_notes = inst.notes
        # ctrl.number is the number of sustain control. If you want to know abour the number type of control,
        # see https://www.midi.org/specifications-old/item/table-3-control-change-messages-data-bytes-2
        ctrls = _control_preprocess([ctrl for ctrl in inst.control_changes if ctrl.number == 64])
        notes += _note_preprocess(ctrls, inst_notes)

    dnotes = _divide_note(notes)

    # print(dnotes)
    dnotes.sort(key=lambda x: x.time)
    # print('sorted:')
    # print(dnotes)
    cur_time = 0
    cur_vel = 0
    for snote in dnotes:
        events += _make_time_sift_events(prev_time=cur_time, post_time=snote.time)
        events += _snote2events(snote=snote, prev_vel=cur_vel)
        # events += _make_time_sift_events(prev_time=cur_time, post_time=snote.time)

        cur_time = snote.time
        cur_vel = snote.velocity

    return [e.to_int() for e in events]

def printEncodedMIDIJupyter(file):
    print(file)
    print(encode_midi_jupyter(file))

In [75]:
printEncodedMIDIJupyter(classic_piano_data[0])

/data/classic_piano/scn15_7_format0.mid
[355, 335, 368, 60, 330, 188, 367, 41, 370, 65, 355, 305, 362, 53, 362, 60, 361, 57, 361, 48, 355, 341, 367, 64, 256, 193, 294, 192, 368, 65, 296, 193, 368, 69, 296, 181, 188, 369, 72, 302, 369, 77, 287, 363, 46, 256, 197, 200, 270, 364, 53, 258, 174, 266, 205, 258, 169, 185, 176, 365, 65, 257, 365, 62, 368, 77, 355, 330, 365, 60, 256, 365, 76, 256, 190, 205, 300, 188, 364, 58, 256, 204, 365, 74, 308, 181, 186, 366, 72, 364, 57, 364, 48, 256, 202, 306, 200, 365, 77, 304, 193, 185, 176, 366, 64, 365, 48, 365, 58, 366, 67, 256, 205, 297, 186, 195, 365, 57, 366, 69, 298, 185, 197, 368, 70, 366, 55, 298, 198, 366, 74, 301, 192, 176, 183, 367, 60, 365, 53, 365, 45, 367, 65, 256, 202, 303, 181, 173, 193, 365, 55, 367, 67, 365, 48, 367, 64, 303, 183, 195, 176, 192, 368, 69, 368, 65, 366, 57, 366, 53, 308, 197, 193, 365, 72, 308, 188, 185, 181, 365, 55, 367, 60, 367, 64, 367, 67, 365, 48, 256, 200, 303, 176, 368, 50, 296, 178, 367, 48, 293, 176, 367, 46,

In [76]:
printEncodedMIDIJupyter(midi_data[0])

/notebooks/src/out/midi/liquidmind-10-thejoyofquietpt2.wav.mid
[]


What appears to be happening is that the pre-processing step excepts the MIDI files to contain sustain notes. Our dataset has no sustain notes.

Let's try and return the raw notes, _if_ no sustain notes are found:

In [78]:
import pretty_midi

midi_file_one = pretty_midi.PrettyMIDI(midi_file=midi_data[0])
piano_file_one = pretty_midi.PrettyMIDI(midi_file=classic_piano_data[0])

from midi_processor.processor import _control_preprocess, _note_preprocess, _divide_note, _make_time_sift_events, _snote2events

def _note_preprocess_jupyter(susteins, notes):
    note_stream = []

    for sustain in susteins:
        for note_idx, note in enumerate(notes):
            if note.start < sustain.start:
                note_stream.append(note)
            elif note.start > sustain.end:
                notes = notes[note_idx:]
                sustain.transposition_notes()
                break
            else:
                sustain.add_managed_note(note)

    for sustain in susteins:
        note_stream += sustain.managed_notes

    note_stream.sort(key= lambda x: x.start)
    return note_stream

def encode_midi_jupyter(file_path):
    events = []
    notes = []
    mid = pretty_midi.PrettyMIDI(midi_file=file_path)

    for inst in mid.instruments:
        inst_notes = inst.notes
        # ctrl.number is the number of sustain control. If you want to know abour the number type of control,
        # see https://www.midi.org/specifications-old/item/table-3-control-change-messages-data-bytes-2
        ctrls_for_preprocessing = [ctrl for ctrl in inst.control_changes if ctrl.number == 64]
        if len(ctrls_for_preprocessing) > 0:
            ctrls = _control_preprocess(ctrls_for_preprocessing)
            notes += _note_preprocess(ctrls, inst_notes)
        else:
            notes = inst_notes

    dnotes = _divide_note(notes)

    dnotes.sort(key=lambda x: x.time)
    cur_time = 0
    cur_vel = 0
    for snote in dnotes:
        events += _make_time_sift_events(prev_time=cur_time, post_time=snote.time)
        events += _snote2events(snote=snote, prev_vel=cur_vel)

        cur_time = snote.time
        cur_vel = snote.velocity

    return [e.to_int() for e in events]

def printEncodedMIDIJupyter(file):
    print(file)
    print(encode_midi_jupyter(file))

In [79]:
printEncodedMIDIJupyter(midi_data[0])

/notebooks/src/out/midi/liquidmind-10-thejoyofquietpt2.wav.mid
[261, 361, 55, 261, 362, 60, 268, 183, 362, 55, 361, 52, 268, 188, 362, 60, 258, 183, 363, 55, 271, 183, 362, 55, 262, 188, 363, 60, 271, 183, 363, 55, 268, 188, 364, 60, 261, 183, 364, 55, 258, 188, 364, 60, 274, 188, 363, 60, 265, 180, 364, 52, 261, 188, 364, 60, 290, 183, 188, 364, 60, 365, 55, 268, 188, 364, 60, 265, 183, 366, 55, 261, 188, 365, 60, 265, 183, 366, 55, 265, 188, 365, 60, 268, 188, 367, 60, 265, 188, 366, 60, 261, 183, 366, 55, 268, 188, 366, 60, 258, 183, 366, 55, 271, 183, 367, 55, 261, 188, 367, 60, 271, 183, 188, 366, 55, 366, 60, 271, 183, 367, 55, 261, 188, 368, 60, 265, 183, 367, 55, 258, 188, 366, 60, 265, 183, 188, 366, 60, 367, 55, 265, 188, 368, 60, 261, 183, 368, 55, 265, 188, 367, 60, 258, 183, 366, 55, 268, 188, 367, 60, 258, 183, 367, 55, 268, 183, 367, 55, 261, 188, 368, 60, 268, 183, 188, 367, 55, 366, 60, 268, 183, 188, 367, 55, 367, 60, 268, 183, 369, 55, 261, 188, 367, 60, 262, 188, 36

Now we're getting data! Let's walk it back up the chain and try preprocessing our data with the fix that handles empty sustain notes:

In [86]:
from preprocess import preprocess_midi
import os

def printPreprocessedData(file):
    print(file)
    print(preprocess_midi(file))    

classic_piano_data = ['{folder}/{file}'.format(folder=CLASSIC_PIANO_DATA, file=file) for file in os.listdir(CLASSIC_PIANO_DATA)]
midi_data = [os.path.abspath('{folder}/{file}'.format(folder=MIDI, file=file)) for file in os.listdir(MIDI)]

In [87]:
printPreprocessedData(classic_piano_data[2])

/data/classic_piano/schuim-4_format0.mid
[366, 83, 366, 44, 265, 364, 87, 211, 264, 366, 83, 172, 264, 364, 80, 265, 208, 366, 80, 366, 63, 364, 59, 364, 56, 265, 211, 364, 83, 265, 208, 366, 80, 265, 364, 75, 265, 203, 366, 75, 265, 208, 364, 80, 265, 203, 366, 75, 265, 364, 71, 265, 199, 366, 71, 265, 203, 364, 75, 265, 199, 366, 71, 265, 364, 68, 265, 196, 366, 68, 265, 199, 364, 71, 265, 196, 366, 68, 265, 191, 364, 63, 265, 191, 364, 63, 363, 58, 363, 67, 365, 70, 363, 51, 260, 215, 187, 184, 211, 208, 203, 199, 196, 274, 191, 186, 195, 198, 179, 279, 366, 83, 366, 44, 265, 211, 364, 87, 265, 172, 366, 83, 265, 364, 80, 265, 208, 366, 80, 366, 63, 364, 59, 364, 56, 265, 211, 364, 83, 265, 208, 366, 80, 265, 364, 75, 265, 203, 366, 75, 265, 208, 364, 80, 265, 203, 366, 75, 265, 364, 71, 265, 199, 366, 71, 265, 203, 364, 75, 265, 199, 366, 71, 265, 364, 68, 265, 196, 366, 68, 265, 199, 364, 71, 265, 196, 366, 68, 265, 191, 364, 63, 265, 191, 363, 58, 363, 51, 363, 67, 365, 70, 364, 

In [88]:
printEncodedMIDIJupyter(midi_data[0])

/notebooks/src/out/midi/liquidmind-10-thejoyofquietpt2.wav.mid
[261, 361, 55, 261, 362, 60, 268, 183, 362, 55, 361, 52, 268, 188, 362, 60, 258, 183, 363, 55, 271, 183, 362, 55, 262, 188, 363, 60, 271, 183, 363, 55, 268, 188, 364, 60, 261, 183, 364, 55, 258, 188, 364, 60, 274, 188, 363, 60, 265, 180, 364, 52, 261, 188, 364, 60, 290, 183, 188, 364, 60, 365, 55, 268, 188, 364, 60, 265, 183, 366, 55, 261, 188, 365, 60, 265, 183, 366, 55, 265, 188, 365, 60, 268, 188, 367, 60, 265, 188, 366, 60, 261, 183, 366, 55, 268, 188, 366, 60, 258, 183, 366, 55, 271, 183, 367, 55, 261, 188, 367, 60, 271, 183, 188, 366, 55, 366, 60, 271, 183, 367, 55, 261, 188, 368, 60, 265, 183, 367, 55, 258, 188, 366, 60, 265, 183, 188, 366, 60, 367, 55, 265, 188, 368, 60, 261, 183, 368, 55, 265, 188, 367, 60, 258, 183, 366, 55, 268, 188, 367, 60, 258, 183, 367, 55, 268, 183, 367, 55, 261, 188, 368, 60, 268, 183, 188, 367, 55, 366, 60, 268, 183, 188, 367, 55, 367, 60, 268, 183, 369, 55, 261, 188, 367, 60, 262, 188, 36

Now a single file is being processed correctly. Let's try the whole dataset.

In [91]:
from preprocess import preprocess_midi_files_under

preprocess_midi_files_under(CLASSIC_PIANO_DATA, '/tmp/data/classic_piano_preprocessed')
preprocess_midi_files_under(MIDI, '/tmp/data/midi')

MIDI Paths: 100%|██████████| 156/156 [00:26<00:00,  5.89it/s]


In [92]:
from data import Data
import tensorflow as tf

batch_size = 2
max_seq = 1024

input_paths = [
    '/tmp/data/classic_piano_preprocessed',
    '/tmp/data/midi',    
]

for input_path in input_paths:
    print('\n\nInput path: {}'.format(input_path))
    dataset = Data(input_path)
    batch_x, batch_y = dataset.slide_seq2seq_batch(batch_size, max_seq)
    print('batch_x')
    print(batch_x.shape)
    print(batch_x[0:5])
    print('min', tf.math.reduce_min(batch_x, axis=None, keepdims=False, name=None))
    print('max', tf.math.reduce_max(batch_x, axis=None, keepdims=False, name=None))
    print('mean', tf.math.reduce_mean(batch_x, axis=None, keepdims=False, name=None))
    print('batch_y')
    print(batch_y.shape)
    print(batch_y)
    print('min', tf.math.reduce_min(batch_y, axis=None, keepdims=False, name=None))
    print('max', tf.math.reduce_max(batch_y, axis=None, keepdims=False, name=None))
    print('mean', tf.math.reduce_mean(batch_y, axis=None, keepdims=False, name=None))



Input path: /tmp/data/classic_piano_preprocessed
batch_x
(2, 1024)
[[256 368  45 ... 371  73 213]
 [372  64 369 ... 257 178 264]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(377, shape=(), dtype=int64)
mean tf.Tensor(221, shape=(), dtype=int64)
batch_y
(2, 1024)
[[368  45 256 ...  73 213 376]
 [ 64 369  60 ... 178 264 364]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(377, shape=(), dtype=int64)
mean tf.Tensor(221, shape=(), dtype=int64)


Input path: /tmp/data/midi
batch_x
(2, 1024)
[[183 364  50 ...  57 268 190]
 [366  59 369 ... 369  57 371]]
min tf.Tensor(29, shape=(), dtype=int64)
max tf.Tensor(374, shape=(), dtype=int64)
mean tf.Tensor(216, shape=(), dtype=int64)
batch_y
(2, 1024)
[[364  50 364 ... 268 190 367]
 [ 59 369  64 ...  57 371  60]]
min tf.Tensor(29, shape=(), dtype=int64)
max tf.Tensor(374, shape=(), dtype=int64)
mean tf.Tensor(216, shape=(), dtype=int64)


This looks much better! Let's give training a shot.

In [93]:
from train import train
train(
    '/tmp/data/midi',
    './save/midi',
    log_dir='/tmp/logs/midi-preprocess'
)

<class Data has "156" files>


****
batch_x
(2, 1024)
[[100 261 187 ... 261 187 179]
 [369  66 268 ... 271 194 369]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(375, shape=(), dtype=int64)
mean tf.Tensor(218, shape=(), dtype=int64)
batch_y
(2, 1024)
[[261 187 228 ... 187 179 185]
 [ 66 268 182 ... 194 369  66]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(375, shape=(), dtype=int64)
mean tf.Tensor(218, shape=(), dtype=int64)

Epoch/Batch: 0/0
Train >>>> Loss: 6.20663, Accuracy: 0.005859375
Eval >>>> Loss: 6.14138, Accuracy: 0.0029296875
****
batch_x
(2, 1024)
[[ 62 287 171 ... 185 178 183]
 [ 40 277 175 ...  59 258 175]]
min tf.Tensor(40, shape=(), dtype=int64)
max tf.Tensor(369, shape=(), dtype=int64)
mean tf.Tensor(214, shape=(), dtype=int64)
batch_y
(2, 1024)
[[287 171 365 ... 178 183 190]
 [277 175 168 ... 258 175 365]]
min tf.Tensor(40, shape=(), dtype=int64)
max tf.Tensor(369, shape=(), dtype=int64)
mean tf.Tensor(215, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[268 366  50 ... 388 388

****
batch_x
(2, 1024)
[[265 192 368 ... 185 370  57]
 [258 196 371 ... 187 370  59]]
min tf.Tensor(54, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(220, shape=(), dtype=int64)
batch_y
(2, 1024)
[[192 368  64 ... 370  57 271]
 [196 371  68 ... 370  59 268]]
min tf.Tensor(54, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(220, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[186 370  58 ... 271 188 370]
 [ 36 345 171 ...  36 261 194]]
min tf.Tensor(35, shape=(), dtype=int64)
max tf.Tensor(374, shape=(), dtype=int64)
mean tf.Tensor(220, shape=(), dtype=int64)
batch_y
(2, 1024)
[[370  58 265 ... 188 370  60]
 [345 171 287 ... 261 194 258]]
min tf.Tensor(35, shape=(), dtype=int64)
max tf.Tensor(374, shape=(), dtype=int64)
mean tf.Tensor(220, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[180 365  52 ... 175 365  47]
 [173 258 188 ... 366  64 368]]
min tf.Tensor(36, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=

****
batch_x
(2, 1024)
[[258 192 371 ... 194 185 190]
 [274 180 367 ... 257 173 355]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(217, shape=(), dtype=int64)
batch_y
(2, 1024)
[[192 371  64 ... 185 190 370]
 [180 367  52 ... 173 355 302]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(217, shape=(), dtype=int64)

Epoch/Batch: 0/39
Train >>>> Loss: 5.83496, Accuracy: 0.0032589933834969997
Eval >>>> Loss: 5.76837, Accuracy: 0.0037286931183189154
****
batch_x
(2, 1024)
[[258 187 366 ...  63 258 199]
 [364  56 268 ... 363  60 268]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(221, shape=(), dtype=int64)
batch_y
(2, 1024)
[[187 366  59 ... 258 199 265]
 [ 56 268 370 ...  60 268 200]]
min tf.Tensor(33, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(221, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[285

****
batch_x
(2, 1024)
[[190 368  62 ... 258 188 367]
 [173 185 367 ... 366  50 287]]
min tf.Tensor(36, shape=(), dtype=int64)
max tf.Tensor(370, shape=(), dtype=int64)
mean tf.Tensor(216, shape=(), dtype=int64)
batch_y
(2, 1024)
[[368  62 258 ... 188 367  60]
 [185 367  57 ...  50 287 197]]
min tf.Tensor(36, shape=(), dtype=int64)
max tf.Tensor(370, shape=(), dtype=int64)
mean tf.Tensor(216, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[ 58 370  67 ... 371  69 370]
 [268 182 368 ... 370  63 258]]
min tf.Tensor(41, shape=(), dtype=int64)
max tf.Tensor(373, shape=(), dtype=int64)
mean tf.Tensor(218, shape=(), dtype=int64)
batch_y
(2, 1024)
[[370  67 274 ...  69 370  64]
 [182 368  54 ...  63 258 182]]
min tf.Tensor(41, shape=(), dtype=int64)
max tf.Tensor(373, shape=(), dtype=int64)
mean tf.Tensor(218, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[355 355 285 ... 388 388 388]
 [ 39 258 169 ... 261 205 363]]
min tf.Tensor(22, shape=(), dtype=int64)
max tf.Tensor(390, shape=(), dtype=

...

****
batch_x
(2, 1024)
[[ 61 265 181 ...  49 268 170]
 [265 195 370 ... 197 370  69]]
min tf.Tensor(25, shape=(), dtype=int64)
max tf.Tensor(371, shape=(), dtype=int64)
mean tf.Tensor(216, shape=(), dtype=int64)
batch_y
(2, 1024)
[[265 181 368 ... 268 170 182]
 [195 370  67 ... 370  69 265]]
min tf.Tensor(25, shape=(), dtype=int64)
max tf.Tensor(371, shape=(), dtype=int64)
mean tf.Tensor(216, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[368  49 261 ... 170 364  42]
 [188 364  60 ... 196 261 365]]
min tf.Tensor(37, shape=(), dtype=int64)
max tf.Tensor(370, shape=(), dtype=int64)
mean tf.Tensor(222, shape=(), dtype=int64)
batch_y
(2, 1024)
[[ 49 261 170 ... 364  42 274]
 [364  60 365 ... 261 365  68]]
min tf.Tensor(37, shape=(), dtype=int64)
max tf.Tensor(370, shape=(), dtype=int64)
mean tf.Tensor(222, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[367  52 368 ... 312 176 368]
 [258 191 193 ...  54 261 183]]
min tf.Tensor(36, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=

****
batch_x
(2, 1024)
[[370  64 368 ... 368  62 293]
 [262 192 369 ... 173 189 369]]
min tf.Tensor(45, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(221, shape=(), dtype=int64)
batch_y
(2, 1024)
[[ 64 368  67 ...  62 293 193]
 [192 369  64 ... 189 369  61]]
min tf.Tensor(45, shape=(), dtype=int64)
max tf.Tensor(372, shape=(), dtype=int64)
mean tf.Tensor(221, shape=(), dtype=int64)

Epoch/Batch: 4/39
Train >>>> Loss: 4.10165, Accuracy: 0.055357467383146286
Eval >>>> Loss: 4.64533, Accuracy: 0.0557861328125
****
batch_x
(2, 1024)
[[258 202 198 ... 258 183 190]
 [348 172 355 ...  35 354 365]]
min tf.Tensor(21, shape=(), dtype=int64)
max tf.Tensor(371, shape=(), dtype=int64)
mean tf.Tensor(217, shape=(), dtype=int64)
batch_y
(2, 1024)
[[202 198 369 ... 183 190 187]
 [172 355 355 ... 354 365  38]]
min tf.Tensor(21, shape=(), dtype=int64)
max tf.Tensor(371, shape=(), dtype=int64)
mean tf.Tensor(217, shape=(), dtype=int64)
****
batch_x
(2, 1024)
[[370  40 26

E0525 20:59:52.233707 140562189899584 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.





Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-93-33c8f1d3e0c7>", line 2, in <module>
    train(
  File "/notebooks/src/MusicTransformer-tensorflow2.0/train.py", line 78, in train
  File "/notebooks/src/MusicTransformer-tensorflow2.0/model.py", line 359, in save
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/network.py", line 1169, in save_weights
    checkpoint_management.update_checkpoint_state_internal(
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/training/checkpoint_management.py", line 226, in update_checkpoint_state_internal
    ckpt = generate_checkpoint_state_proto(
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/training/checkpoint_management.py", line 117, in generate_checkpoint_state_proto
    model_checkpoint_path = os.path.relpath(model_

FileNotFoundError: [Errno 2] No such file or directory

These logs look much better; our training data appears to be tracking much closer to the `classic_piano` dataset:

![New Accuracy](new-accuracy.png)

Loss:

![New Loss](new-loss.png)